## GLAM fit. Value based. Gamma parameter limited to 0 - 1 . (Marius Usher recommendation)

In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# 3.1. Individual GLAM estimation prediction

## Load data

In [4]:
# Load data
sufix = '_individual_Dislike_NoBin_Gamma_01_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Dislike_NoBin_31.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
1,0,1,1,3424,2.30,1.70,0.410720,0.589280
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
3,0,3,0,8144,1.55,2.30,0.592345,0.407655
4,0,4,0,6559,2.00,2.00,0.641717,0.358283


## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


In [6]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
4,0,4,0,6559,2.00,2.00,0.641717,0.358283
6,0,6,1,2470,1.55,1.10,0.329730,0.670270
8,0,8,1,4669,1.55,1.70,0.419431,0.580569
10,0,10,0,3301,1.10,1.10,0.605079,0.394921
12,0,12,1,3117,1.85,1.10,0.506602,0.493398
14,0,14,0,2754,1.25,1.55,0.596632,0.403368
16,0,16,1,3931,1.85,1.70,0.338392,0.661608
18,0,18,0,5674,1.70,1.40,0.410655,0.589345


## Individual GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(0, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_FF2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 31 subjects...


INFO (theano.gof.compilelock): Waiting for existing lock by process '58462' (I am process '58457')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.6.0-x86_64-i386-64bit-i386-3.7.3-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '58462' (I am process '58457')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.6.0-x86_64-i386-64bit-i386-3.7.3-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '58462' (I am process '58457')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.6.0-x86_64-i386-64bit-i386-3.7.3-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '58462' (I am process '58457')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.6.0-x86_64-i3

Fitting 31 model(s) using NUTS...
  Fitting model 1 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [12:35<00:00,  5.50draws/s]
The acceptance probability does not match the target. It is 0.9678570555295564, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 31...


...

  Fitting model 30 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [10:42<00:00,  4.39draws/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 31 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [01:18<00:00, 153.06draws/s]
The acceptance probability does not match the target. It is 0.9022322734837991, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9087966782639275, but should be close to 0.8. Try to increase the number of tuning steps.


/!\ Automatically setting parameter precision...


In [8]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_full'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

,SNR,b,gamma,p_error,s,t0,tau,v
0,46.19,1.0,0.19,0.05,0.003239,[0.0],0.51,0.000059
1,37.97,1.0,0.38,0.05,0.004252,[0.0],0.45,0.000053
2,43.56,1.0,0.29,0.05,0.003889,[0.0],0.01,0.000050
3,61.80,1.0,0.15,0.05,0.004826,[0.0],0.95,0.000078
4,181.86,1.0,0.26,0.05,0.011398,[0.0],1.18,0.000065
5,140.20,1.0,0.01,0.05,0.007892,[0.0],0.01,0.000060
6,137.98,1.0,0.01,0.05,0.010018,[0.0],0.02,0.000063
7,139.68,1.0,0.01,0.05,0.007933,[0.0],0.02,0.000054
8,75.03,1.0,0.33,0.05,0.005674,[0.0],0.42,0.000055
9,146.95,1.0,0.01,0.05,0.007912,[0.0],0.01,0.000071


In [9]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('results/params_estimates/params_glam_FF2019_full'+sufix+'.csv'))

In [10]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_FF2019_full'+ sufix +'.npy'))

Computing WAIC scores for full model...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:168: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.

In [11]:
# Compute WAICs
np.save(str('results/waic/glam_FF2019_full'+ sufix +'.npy'), glam_full.waic)

In [12]:
waic_values = pd.DataFrame(glam_full.waic)
waic_values.to_csv(str('results/params_estimates/waic_estimate'+sufix+'.csv'))

In [13]:
len(glam_full.waic)

31

In [14]:
glam_full.waic

array([[3.67510587e+03, 0.00000000e+00, 1.27991643e+03, 1.00000000e+00],
       [1.77681731e+03, 0.00000000e+00, 3.40834640e+02, 1.00000000e+00],
       [1.16924109e+04, 0.00000000e+00, 5.29046438e+03, 1.00000000e+00],
       [2.79896568e+03, 0.00000000e+00, 8.28909984e+02, 1.00000000e+00],
       [1.56496455e+03, 0.00000000e+00, 2.26515317e+02, 1.00000000e+00],
       [1.09649556e+03, 0.00000000e+00, 1.96003589e+00, 1.00000000e+00],
       [1.08702611e+03, 0.00000000e+00, 1.95131486e+00, 1.00000000e+00],
       [1.08408026e+03, 0.00000000e+00, 2.25676056e+00, 1.00000000e+00],
       [1.11406139e+03, 0.00000000e+00, 1.94196570e+01, 1.00000000e+00],
       [1.08374098e+03, 0.00000000e+00, 1.99816935e+00, 1.00000000e+00],
       [1.06649756e+03, 0.00000000e+00, 1.83220820e+00, 1.00000000e+00],
       [1.07397658e+03, 0.00000000e+00, 1.96972366e+00, 1.00000000e+00],
       [3.80984391e+03, 0.00000000e+00, 1.36193168e+03, 1.00000000e+00],
       [1.14350604e+03, 0.00000000e+00, 1.90753859e

In [15]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'list' object has no attribute 'nchains'

In [16]:
glam_full.loo

AttributeError: 'GLAM' object has no attribute 'loo'

In [17]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,subject,trial,repeat,choice,rt,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,0.0,1.0,3405.0,2.3,0.41072,1.7,0.58928
1,0.0,0.0,1.0,1.0,3291.0,2.3,0.41072,1.7,0.58928
2,0.0,0.0,2.0,0.0,2816.0,2.3,0.41072,1.7,0.58928
3,0.0,0.0,3.0,0.0,2832.0,2.3,0.41072,1.7,0.58928
4,0.0,0.0,4.0,0.0,3003.0,2.3,0.41072,1.7,0.58928


## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
glam.plots_pretty.plot_fit(test_data, [glam_full.prediction]);

#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## [END] 